<a href="https://colab.research.google.com/github/WelfLowe/ML4developers/blob/main/6_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [59]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X.shape[0]

150

In [60]:
X_ = np.matrix.transpose(np.c_[X, np.ones(X.shape[0])])
X_.shape

(5, 150)

In [34]:
Wb = np.random.rand(3,5)
Wb

array([[0.84234022, 0.9382266 , 0.0442454 , 0.35591702, 0.67603667],
       [0.80615816, 0.00661135, 0.17240509, 0.45596478, 0.99787635],
       [0.14911478, 0.27928023, 0.03706293, 0.91536605, 0.11813457]])

In [43]:
y_ = np.matrix.transpose(np.matmul(Wb, X_)) # y = WX+b
y_[0:5,:]

array([[8.38889183, 5.46498279, 2.09106207],
       [7.75131049, 5.30044548, 1.921599  ],
       [7.76606322, 5.12329561, 1.9439258 ],
       [7.59685562, 5.07649968, 1.90849888],
       [8.39848047, 5.38502811, 2.10407862]])

In [53]:
y_hat = np.rint(np.amax(y_, axis = 1))
max_class = lambda x: min(x,2) 
min_class = lambda x: max(x,0) 
y_hat = np.vectorize(max_class)(y_hat)
y_hat = np.vectorize(min_class)(y_hat)
y_hat

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [54]:
cm = confusion_matrix(y, y_hat)
cm

array([[ 0,  0, 50],
       [ 0,  0, 50],
       [ 0,  0, 50]])

In [61]:
def eval_model(X, y, Wb):
  X_ = np.matrix.transpose(np.c_[X, np.ones(X.shape[0])])
  y_ = np.matrix.transpose(np.matmul(Wb, X_)) # y = WX+b
  y_hat = np.rint(np.amax(y_, axis = 1))
  y_hat = np.vectorize(max_class)(y_hat)
  y_hat = np.vectorize(min_class)(y_hat)
  return confusion_matrix(y, y_hat)

eval_model(X, y, Wb)

array([[ 0,  0, 50],
       [ 0,  0, 50],
       [ 0,  0, 50]])

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #random_state=2

In [65]:
eval_model(X_test, y_test, Wb)

array([[ 0,  0, 10],
       [ 0,  0,  9],
       [ 0,  0, 11]])